In [4]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare ensemble outputs with NLDAS data
import os
os.environ["PROJ_LIB"] = '/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/share/proj'

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime

startTime = datetime.datetime.now()

def read_nldas(file_dir, start_yr, end_yr):
    for yr in range(start_yr, end_yr+1):
        nldas_file = 'NLDAS_'+str(yr)+'.nc'
        nldas_path = os.path.join(file_dir, nldas_file)

        f_nldas = xr.open_dataset(nldas_path)
        if yr == start_yr:
            pcp_concat = f_nldas['pcp'].values[:] # (time, y, x). unit: mm/day
            t_mean_concat = f_nldas['t_mean'].values[:] # (time, y, x). unit: degC
            t_range_concat = f_nldas['t_range'].values[:]
            time_concat = f_nldas['time'].values[:]
        else:
            pcp_concat = np.concatenate((pcp_concat, f_nldas['pcp'].values[:]), axis = 0)
            t_mean_concat = np.concatenate((t_mean_concat, f_nldas['t_mean'].values[:]), axis = 0)
            t_range_concat = np.concatenate((t_range_concat, f_nldas['t_range'].values[:]), axis = 0)
            time_concat = np.concatenate((time_concat, f_nldas['time'].values[:]), axis = 0)

    # get time mask from nldas data
    time_obj = pd.to_datetime(time_concat)
    return time_obj, pcp_concat, t_mean_concat, t_range_concat

def read_regress_pop(file_dir, start_yr, end_yr):
    for yr in range(start_yr, end_yr+1):        
        
        filename='regress_ts.%d.nc'%(yr)
        file = os.path.join(file_dir, filename)
        f=xr.open_dataset(file)
        time = f['time'].values[:]
        pop = f['pop'].values[:] # std

        if yr == start_yr:
            time_concat = time
            pop_concat = pop
        else:
            time_concat = np.concatenate((time_concat,time), axis=0) # (time)
            pop_concat = np.concatenate((pop_concat, pop), axis=0) # (time,y,x)
            
    time_concat = pd.to_datetime(time_concat)        
    return time_concat, pop_concat

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx):

    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawstates(linewidth=0.5, linestyle='solid', color='grey')
    m.drawcountries(linewidth=0.5, linestyle='solid', color='k')
    m.drawcoastlines(linewidth=.25, linestyle='solid', color='k')
    return m

def create_colormap(cmap):
    # create a white-cmap linear colormap
    # reference: https://stackoverflow.com/questions/25408393/getting-individual-colors-from-a-color-map-in-matplotlib
    cmap = mpl.cm.get_cmap(cmap) #‘viridis’，'jet' # get the blue color of jet 
    c0 = cmap(0.0)
#     top = mpl.colors.LinearSegmentedColormap.from_list("", ["white",c0])
    top = mpl.colors.LinearSegmentedColormap.from_list("", ["white","white"])

    # combine two liner colormaps to create a
    # reference: https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html
    bottom = mpl.cm.get_cmap(cmap) #'jet','viridis'
    newcolors = np.vstack((top(np.linspace(0, 1, int(256*0.01))),bottom(np.linspace(0, 1, int(256*0.99)))))
    newcmp = mpl.colors.LinearSegmentedColormap.from_list("NewCmap", newcolors)
    return newcmp

# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar.
    source: http://chris35wills.github.io/matplotlib_diverging_colorbar/
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-300, vmax=1000))    
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc_convert')
start_yr = 2016
end_yr = 2016

gridinfo_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')

result_dir = os.path.join(root_dir,'test_uniform_perturb')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)

regress_subforlder = 'gmet_regr'
ens_subforlder = 'gmet_ens_bc'

ens_num = 100
time_format = '%Y-%m-%d'

dpi_value = 150
plot_date_start = '2016-06-01'
plot_date_end = '2016-06-30'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)

output_dir=os.path.join(root_dir, 'scripts/step34_plot_spatial_PoP')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# #======================================================================================================
# print('Read gridinfo mask')
# # get xy mask from gridinfo.nc
# f_gridinfo = xr.open_dataset(gridinfo_file)
# mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
# #data_mask = f_gridinfo['data_mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
# latitude = f_gridinfo['latitude'].values[:]
# longitude = f_gridinfo['longitude'].values[:]

# #======================================================================================================
# # read historical nldas data
# print('Read nldas data')
# time_obj, pcp, tmean, trange=read_nldas(nldas_dir,start_yr, end_yr)
# mask_t  = (time_obj >= plot_date_start_obj) & (time_obj <= plot_date_end_obj) 
# time = time_obj[mask_t]

# # pop binary
# pop_binary = np.where(pcp[mask_t,:,:]>0, 1, 0) #(time, y, x))

# # convert masked values to nan
# pop_binary=np.where(mask_xy==0,np.nan,pop_binary)

# # calculate pop
# pop=np.nanmean(pop_binary,axis=0) 

# del pcp

#======================================================================================================
print('Plot')
# loop through all uniform tests
for test_folder in test_folders[-1:]:
    
    print(test_folder)
    test_dir = os.path.join(result_dir, test_folder)
    
    # read pop from regression results    
    file_dir = os.path.join(test_dir,regress_subforlder)
    time_pop, pop_regres = read_regress_pop(file_dir, start_yr, end_yr)   

    ## part 2. calcualte time-series mean for extract period
    # define plot mask for nldas ensemble
    pop_mask_t = (time_pop>=plot_date_start_obj) & (time_pop<=plot_date_end_obj)
    
    # caluclate time series mean(ny,nx)
    pop_regres_mean = np.nanmean(pop_regres[pop_mask_t,:,:],axis=0) 
    
    # convert masked values to nan
    pop_regres_mean=np.where(mask_xy==0,np.nan,pop_regres_mean)
    
    # calcualte pop differnece between NLDAS and regression
    pop_diff = pop_regres_mean-pop
    
    ## part 3. setup plot colorbar range for the plot_date
    vmin_pop=0
    vmax_pop=1

    lim=max([abs(np.nanmin(pop_diff)),abs(np.nanmax(pop_diff))])
    vmin_pop_diff=(-1)*(lim)
    vmax_pop_diff=lim

    # plot
    nrow = 1 # pop
    ncol = 3 # NLDAS, estiamte, difference   
    fig, ax = plt.subplots(nrow, ncol, figsize=(7.08,7.08*0.85*0.25))
    # figure size (width, height) 
    #constrained_layout=True
    fig.subplots_adjust(left=0.1, bottom=0.1, right=1, top=0.9)#, wspace=None, hspace=None)
    
    llcrnrlon = longitude[0,0]
    urcrnrlon = longitude[-1,-1]
    llcrnrlat = latitude[0,0]
    urcrnrlat = latitude[-1,-1]
    lat_0=0.5*(llcrnrlat+urcrnrlat)
    lon_0=0.5*(llcrnrlon+urcrnrlon)
    (ny,nx)=np.shape(longitude)

    for j in range(ncol):

        # select data for each subplot
        # PCP (1st column, 1st-3st row)
        if j<(ncol-1):
#             newcmp = create_colormap(plt.cm.jet_r)
#             cmap,vmin,vmax=newcmp, vmin_pop, vmax_pop
            cmap,vmin,vmax=plt.cm.jet_r, vmin_pop, vmax_pop
            if j == 0:
                data,title_str=pop,'(a) NLDAS PoP'
            elif j==1:
                data,title_str=pop_regres_mean,'(b) Ensemble PoP'
        elif j==(ncol-1):
            cmap,vmin,vmax=plt.cm.bwr_r, vmin_pop_diff, vmax_pop_diff
            data,title_str=pop_diff,'(c) Ensemble - NLDAS'               

        # plot Basemap
        m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax[j],lat_0,lon_0,ny,nx) # plot Basemap 

        # plot data
        im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)

        # set colorbar
        cbar = fig.colorbar(im,ax=ax[j],pad=0.03,orientation="horizontal")  
        cbar.ax.tick_params(labelsize='xx-small',pad=0.05, length=2)             

        # set title
        ax[j].set_title(title_str, fontsize='xx-small', fontweight='semibold')

    # save plot
    fig.tight_layout(pad=0.2)
    output_filename = test_folder+'.png'
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)

print('Done')


Plot
18212grids


<ipython-input-4-125b9d62abfd>:173: RuntimeWarning: Mean of empty slice
  pop_regres_mean = np.nanmean(pop_regres[pop_mask_t,:,:],axis=0)


Done


In [22]:
np.nanmean(pop_diff)

-0.010152897473142346